In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import PIL
from google.colab.patches import cv2_imshow
import gspread
import torch
import torchvision
import torchvision.transforms as transforms
import sklearn

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
NoFind = 0
Covid = 1
Atelectasis = 3
Cardiomegaly = 4
Effusion = 5
Infiltration = 6
MASS = 7
Nodule = 8
Pneumonia = 9
Pneumothorax = 10
Consolidation = 11
Edema = 12
Emphysema, Fibrosis, Pleural, Hernia, Tuberculosis, Opacity, Other = 13, 14, 15, 16, 17, 18, 19
testing_path = '/content/gdrive/MyDrive/DiseaseDetection/Testing/Images/'

tb_path_pos = '/content/gdrive/MyDrive/DiseaseDetection/TB_Chest_Radiography_Database/Tuberculosis/'
tb_path_neg = '/content/gdrive/MyDrive/DiseaseDetection/TB_Chest_Radiography_Database/Normal/'
tb_data = '/content/gdrive/MyDrive/DiseaseDetection/TB_Chest_Radiography_Database/Tuberculosis.csv'

cov1_path_cov = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/COVID/images/'
cov1_path_vir = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/Viral Pneumonia/images/'
cov1_path_opac = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/Lung_Opacity/images/'
cov1_path_neg = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/Normal/images/'
cov1_data = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/data.csv'

cov2_images = '/content/gdrive/MyDrive/DiseaseDetection/covid-chestxray-dataset-master/images/'
cov2_data = '/content/gdrive/MyDrive/DiseaseDetection/covid-chestxray-dataset-master/Data.csv'

dd_paths = []
for i in range(1, 13):
  x = '/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_0'
  if i < 10:
    x += '0'
  x += str(i) + '/images/'
  dd_paths.append(x)

dd_data = '/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/data.csv'

In [ ]:
tb_df = pd.read_csv(tb_data)
cov1_df = pd.read_csv(cov1_data)
cov2_df = pd.read_csv(cov2_data)
dd_df = pd.read_csv(dd_data)

tb_df.rename(columns = {"FILE NAME": "Name"}, inplace = True)
cov1_df.rename(columns = {"FILE NAME": "Name"}, inplace = True)
cov2_df.rename(columns = {"filename": "Name", 'finding': 'Diagnosis'}, inplace = True)
dd_df.rename(columns = {"Image Index": "Name", 'Finding Labels': 'Diagnosis'}, inplace = True)

def split(x):
  if "|" in x:
    return x.split('|')
  elif "/" in x:
    return x.split('/')
  else:
    if type(x) != list:
      return [x]
    else:
      return x
def organize(x):
  for i in range(len(x)):
    if x[i].lower() == "covid" or x[i].lower() == "covid-19":
      x[i] = Covid
    elif x[i] == "None" or x[i] == "No Finding":
      x[i] = NoFind
    elif x[i] == "Viral":
      x[i] = Pneumonia
    elif x[i] == "Atelectasis":
      x[i] = Atelectasis
    elif x[i] == "Cardiomegaly":
      x[i] = Cardiomegaly
    elif x[i] == "Effusion":
      x[i] = Effusion
    elif x[i] == "Infiltration":
      x[i] = Infiltration
    elif x[i] == "Mass":
      x[i] = MASS
    elif x[i] == "Nodule":
      x[i] = Nodule
    elif x[i] == "Pneumonia":
      x[i] = Pneumonia
    elif x[i] == "Pneumothorax":
      x[i] = Pneumothorax
    elif x[i] == "Consolidation":
      x[i] = Consolidation
    elif x[i] == "Edema":
      x[i] = Edema
    elif x[i] == "Emphysema":
      x[i] = Emphysema
    elif x[i] == "Fibrosis":
      x[i] = Fibrosis
    elif x[i] == "Pleural_Thickening":
      x[i] = Pleural
    elif x[i] == "Hernia":
      x[i] = Hernia
    elif x[i] == "Tuberculosis":
      x[i] = Tuberculosis
    elif x[i] == "Opacity":
      x[i] = Opacity
    else:
      x[i] = Other
  return x
def cov1_modify(x):
  if "NORMAL" in x:
    return x.capitalize() + '.png'
  return x + '.png'

def tb_modify(x):
  if "Normal" in x:
    return "Normal TB-" + x.split("-")[1] + '.png'
  return x + ".png"
cov1_df["Name"] = cov1_df["Name"].apply(cov1_modify)
tb_df["Name"] = tb_df["Name"].apply(tb_modify)


df = pd.concat([dd_df, cov2_df, cov1_df, tb_df], ignore_index=True)
df["Diagnosis"] = df["Diagnosis"].apply(split)
df["Diagnosis"] = df["Diagnosis"].apply(organize)
df.rename(columns = {"Name": "Image Name", 'Diagnosis': 'Classification'}, inplace = True)

In [ ]:
temp = df["Classification"].copy()
labels = {df["Image Name"][i]: temp[i][0] for i in range(len(df))}
X_test = []

In [ ]:
import random
for i in os.listdir(tb_path_pos): 
  if random.randint(1, 20) == 1:
    curr = Image.open(tb_path_pos + i)
    curr = curr.resize((224,224))
    curr = curr.convert("L")
    curr = np.stack((curr,)*3)
    curr = np.transpose(curr, (1, 2, 0))
    curr = PIL.Image.fromarray(curr)
    X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [ ]:
for i in os.listdir(tb_path_neg):
  if random.randint(1, 20) == 1:
    curr = Image.open(tb_path_neg + i)
    curr = curr.resize((224,224))
    curr = curr.convert("L")
    curr = np.stack((curr,)*3)
    curr = np.transpose(curr, (1, 2, 0))
    curr = PIL.Image.fromarray(curr)
    X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [ ]:
for i in os.listdir(cov1_path_neg):
  if "(" not in i:
    if random.randint(1, 20) == 1:
      curr = Image.open(cov1_path_neg + i)
      curr = curr.resize((224,224))
      curr = curr.convert("L")
      curr = np.stack((curr,)*3)
      curr = np.transpose(curr, (1, 2, 0))
      curr = PIL.Image.fromarray(curr)
      X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [ ]:
for i in os.listdir(cov1_path_opac):
  if random.randint(1, 20) == 1:
    curr = Image.open(cov1_path_opac + i)
    curr = curr.resize((224,224))
    curr = curr.convert("L")
    curr = np.stack((curr,)*3)
    curr = np.transpose(curr, (1, 2, 0))
    curr = PIL.Image.fromarray(curr)
    X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [ ]:
for i in os.listdir(cov1_path_vir):
  if random.randint(1, 20) == 1:
    curr = Image.open(cov1_path_vir + i)
    curr = curr.resize((224,224))
    curr = curr.convert("L")
    curr = np.stack((curr,)*3)
    curr = np.transpose(curr, (1, 2, 0))
    curr = PIL.Image.fromarray(curr)
    X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [ ]:
for i in os.listdir(cov1_path_cov):
  if "(" not in i:
    if random.randint(1, 20) == 1:
      curr = Image.open(cov1_path_cov + i)
      curr = curr.resize((224,224))
      curr = curr.convert("L")
      curr = np.stack((curr,)*3)
      curr = np.transpose(curr, (1, 2, 0))
      curr = PIL.Image.fromarray(curr)
      X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [ ]:
import time
start = time.time()
for i in dd_paths:
  for j in os.listdir(i):
    if random.randint(1, 25) == 1:
      curr = Image.open(i + j)
      curr = curr.resize((224,224))
      curr = curr.convert("L")
      curr = np.stack((curr,)*3)
      curr = np.transpose(curr, (1, 2, 0))
      curr = PIL.Image.fromarray(curr)
      X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[j]))
  print(i, time.time() - start)

/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_001/images/ 141.62361454963684
/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_002/images/ 405.93778014183044
/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_003/images/ 694.7787289619446
/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_004/images/ 976.9530940055847
/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_005/images/ 1259.7463142871857
/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_006/images/ 1532.8629522323608
/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_007/images/ 1806.760930776596
/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_008/images/ 2103.0510246753693
/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_009/images/ 2374.6122653484344
/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_010/images/ 2678.1200416088104
/content/gdrive/MyDrive/DiseaseDetection/DD Full Data

In [ ]:
for i in os.listdir(cov2_images):
  if random.randint(1, 20) == 1:
    curr = Image.open(cov2_images + i)
    curr = curr.resize((224,224))
    curr = curr.convert("L")
    curr = np.stack((curr,)*3)
    curr = np.transpose(curr, (1, 2, 0))
    curr = PIL.Image.fromarray(curr)
    if i in labels:
      X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [ ]:
from sklearn.model_selection import train_test_split

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
x_train, x_test, y_train, y_test = train_test_split([i[0] for i in X_test], [i[1] for i in X_test], test_size = 0.2)
new_x_train = []
for i in range(len(x_train)):
  new_x_train.append([x_train[i], y_train[i]])
new_x_test = []
for i in range(len(x_test)):
  new_x_test.append([x_test[i], y_test[i]])

train_loader = torch.utils.data.DataLoader(new_x_train, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(new_x_test, batch_size=len(new_x_test), shuffle=True)

In [ ]:
from torch import nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 19):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
num_classes = 10
num_epochs = 5
batch_size = 16
learning_rate = 0.01

model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)  

# Train the model
total_step = len(train_loader)

In [ ]:
import gc
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images.float())
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()

    print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item()))
            
    # Validation
    # with torch.no_grad():
    #     correct = 0
    #     total = 0
    #     for images, labels in valid_loader:
    #         images = images.to(device)
    #         labels = labels.to(device)
    #         outputs = model(images)
    #         _, predicted = torch.max(outputs.data, 1)
    #         total += labels.size(0)
    #         correct += (predicted == labels).sum().item()
    #         del images, labels, outputs
    
    #     print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))

Epoch [1/5], Loss: 1.4277
Epoch [2/5], Loss: 1.6370
Epoch [3/5], Loss: 1.9519
Epoch [4/5], Loss: 2.4378
Epoch [5/5], Loss: 1.3208


In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the test images: {} %'.format(100 * correct / total))   